In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
features_df = pd.read_csv("../../data/gene_df_128.csv")
features_df.head()
label_df = pd.read_csv("../../data/final_labels.csv")
label_df.index = label_df['File ID']
labels = label_df.loc[features_df['file_name']]['LABEL']
labels_more = label_df.loc[features_df['file_name']]['final_label']
projects = label_df.loc[features_df['file_name']]['Project ID']

Using t-SNE


In [ ]:
X = features_df.iloc[:,1:]
Y = labels
tsne = TSNE(n_components=2, verbose=1, random_state=14)
z = tsne.fit_transform(X)
df = pd.DataFrame()
df["y"] = Y
df["y_more"] = labels_more
df["projects"] = projects
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]


pt1 = sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df)
pt1.set(title="t-SNE: 128 Latent Gene")
pt1.get_figure().savefig("../../Figures/tsne_gene_1.png")

Using PCA


In [ ]:
X = features_df.iloc[:,1:]
Y = labels
pca = PCA(n_components=2)
z = pca.fit_transform(X)
df = pd.DataFrame()
df["y"] = Y
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

pt1 = sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df)
pt1.set(title="PCA: 128 Latent Gene")
pt1.get_figure().savefig("../../Figures/pca_gene.png")

In [ ]:
pca.explained_variance_ratio_.cumsum()


10 fold cross validation


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from mlmodels import cross_validate_custom
VC = VotingClassifier(estimators=[('SVM', SVC(kernel = 'rbf', probability=True)),
 ('RF', RandomForestClassifier(n_estimators=100))],voting='soft')

estimators = [SVC(kernel='rbf',probability=True), RandomForestClassifier(n_estimators=100),VC, LogisticRegression()]
result = []
for estimator in estimators:
    result.append(cross_validate_custom(X,Y,estimator))
result = pd.DataFrame(result,index = ['SVM','RF','Voting','LR'])
result